In [1]:
!pip install pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a9c30aff9e46b3fd1588f9f2ddb7cc461784967b01b71938cf026eb3108ca06c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
mkdir linkage

In [3]:
cd linkage

/content/linkage


In [4]:
!curl -L -o donation.zip https://bit.ly/1Aoywaq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   163  100   163    0     0   1275      0 --:--:-- --:--:-- --:--:--  1283
100 53.8M    0 53.8M    0     0  27.2M      0 --:--:--  0:00:01 --:--:-- 30.2M


In [5]:
!unzip donation.zip

Archive:  donation.zip
 extracting: block_10.zip            
 extracting: block_1.zip             
 extracting: block_2.zip             
 extracting: block_3.zip             
 extracting: block_4.zip             
 extracting: block_5.zip             
 extracting: block_6.zip             
 extracting: block_7.zip             
 extracting: block_8.zip             
 extracting: block_9.zip             
  inflating: documentation           
  inflating: frequencies.csv         


In [6]:
!unzip 'block_*.zip'

Archive:  block_7.zip
  inflating: block_7.csv             

Archive:  block_8.zip
  inflating: block_8.csv             

Archive:  block_10.zip
  inflating: block_10.csv            

Archive:  block_3.zip
  inflating: block_3.csv             

Archive:  block_2.zip
  inflating: block_2.csv             

Archive:  block_1.zip
  inflating: block_1.csv             

Archive:  block_5.zip
  inflating: block_5.csv             

Archive:  block_4.zip
  inflating: block_4.csv             

Archive:  block_6.zip
  inflating: block_6.csv             

Archive:  block_9.zip
  inflating: block_9.csv             

10 archives were successfully processed.


In [7]:
prev = spark.read.csv("/content/linkage/block*.csv")
prev

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string]

In [8]:
prev.show(2)

+----+----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| _c0| _c1|         _c2|         _c3|         _c4|         _c5|    _c6|   _c7|   _c8|   _c9|   _c10|    _c11|
+----+----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
|id_1|id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
|3148|8326|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
+----+----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
only showing top 2 rows



In [9]:
parsed = spark.read.option("header", "true").option("nullValue", "?").option("inferSchema", "true").csv("/content/linkage/block*.csv")

In [10]:
parsed.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [11]:
from pyspark.sql.types import *


schema = StructType([
    StructField("id_1", IntegerType(), False),
    StructField("id_2", IntegerType(), False),
    StructField("cmp_fname_c1", DoubleType(), False)])

spark.read.schema(schema).csv("/content/linkage/block*.csv").printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)



In [12]:
schema = "id_1 INT, id_2 INT, cmp_fname_c1 DOUBLE"

spark.read.schema(schema).csv("/content/linkage/block*.csv").printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)



In [13]:
d1 = spark.read.format("json").load("/content/json_files/Previsao.json")

In [14]:
d1

DataFrame[_corrupt_record: string]

In [15]:
d2 = spark.read.json("/content/json_files/Previsao.json")

In [16]:
d2

DataFrame[_corrupt_record: string]

In [17]:
d1.write.format("parquet").save("/content/parquet_files/Previsao.json")

AnalysisException: Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().

In [ ]:
d1.write.parquet("/content/parquet_files/Previsao.parquet")

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/content/parquet_files/Previsao.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

In [ ]:
d2.write.format("parquet").mode("overwrite").save("/content/parquet_files/Previsao.parquet")

AnalysisException: Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().

In [18]:
parsed.first()

Row(id_1=3148, id_2=8326, cmp_fname_c1=1.0, cmp_fname_c2=None, cmp_lname_c1=1.0, cmp_lname_c2=None, cmp_sex=1, cmp_bd=1, cmp_bm=1, cmp_by=1, cmp_plz=1, is_match=True)

In [19]:
parsed.count()

5749132

In [20]:
parsed.colletc()

AttributeError: 'DataFrame' object has no attribute 'colletc'

In [21]:
parsed.write.format("parquet").("content/parquet_files/Previsao.parquet")

SyntaxError: invalid syntax (<ipython-input-21-252f80afb9d6>, line 1)

In [22]:
parsed.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [23]:
parsed.show(5)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|14055|94934|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|33948|34740|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|  946|71870|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|64880|71676|         1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--

In [24]:
parsed.count()

5749132

In [25]:
parsed.cache()

DataFrame[id_1: int, id_2: int, cmp_fname_c1: double, cmp_fname_c2: double, cmp_lname_c1: double, cmp_lname_c2: double, cmp_sex: int, cmp_bd: int, cmp_bm: int, cmp_by: int, cmp_plz: int, is_match: boolean]

In [26]:
from pyspark.sql.functions import col


parsed.groupBy("is_match").count().orderBy(col("count").desc()).show()

+--------+-------+
|is_match|  count|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



In [27]:
from pyspark.sql.functions import avg, stddev


parsed.agg(avg("cmp_sex"), stddev("cmp_sex")).show()

+-----------------+-------------------+
|     avg(cmp_sex)|    stddev(cmp_sex)|
+-----------------+-------------------+
|0.955001381078048|0.20730111116897443|
+-----------------+-------------------+



In [28]:
parsed.createOrReplaceTempView("linkage")

In [29]:
spark.sql("""
  SELECT is_match, COUNT(*) AS cnt
  FROM linkage
  GROUP BY is_match
  ORDER BY cnt DESC
""").show()

+--------+-------+
|is_match|    cnt|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



In [30]:
spark_session = SparkSession.builder.master("local[4]").enableHiveSupport().getOrCreate()

In [31]:
summary = parsed.describe()

In [32]:
summary.show()

+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|summary|              id_1|              id_2|      cmp_fname_c1|      cmp_fname_c2|       cmp_lname_c1|       cmp_lname_c2|            cmp_sex|             cmp_bd|             cmp_bm|            cmp_by|            cmp_plz|
+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|  count|           5749132|           5749132|           5748125|            103698|            5749132|               2464|            5749132|            5748337|            5748337|           5748337|            5736289|
|   mean| 33324.48559643438| 66587.43558331935|0.7129024704425707| 0.900017671890335|0.3156278193076

In [33]:
summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

+-------+------------------+------------------+
|summary|      cmp_fname_c1|      cmp_fname_c2|
+-------+------------------+------------------+
|  count|           5748125|            103698|
|   mean|0.7129024704425707| 0.900017671890335|
| stddev|0.3887583596162788|0.2713176105782331|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+



In [34]:
# SQL
matches = parsed.where("is_match = true")
match_summary = matches.describe()

In [56]:
# PySpark
misses = parsed.filter(col("is_match") == False)
miss_summary = misses.describe()

In [38]:
summary_p = summary.toPandas()

In [40]:
summary_p.head()

,summary,id_1,id_2,cmp_fname_c1,cmp_fname_c2,cmp_lname_c1,cmp_lname_c2,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz
0,count,5749132,5749132,5748125,103698,5749132,2464,5749132,5748337,5748337,5748337,5736289
1,mean,33324.48559643438,66587.43558331935,0.7129024704425707,0.900017671890335,0.31562781930763056,0.31841283153174366,0.955001381078048,0.22446526708507172,0.48885529849763504,0.2227485966810923,0.00552866147434343
2,stddev,23659.859374487987,23620.487613269706,0.3887583596162788,0.2713176105782331,0.3342336339615803,0.3685670662006655,0.20730111116897443,0.41722972238461925,0.4998758236779003,0.4160909629831711,0.07414914925420013
3,min,1,6,0.0,0.0,0.0,0.0,0,0,0,0,0
4,max,99980,100000,1.0,1.0,1.0,1.0,1,1,1,1,1


In [41]:
summary_p.shape

(5, 12)

In [42]:
summary_p = summary_p.set_index("summary").transpose().reset_index()

In [43]:
summary_p = summary_p.rename(columns={"index":"field"})

In [44]:
summary_p = summary_p.rename_axis(None, axis=1)

In [45]:
summary_p.shape

(11, 6)

In [46]:
summaryT = spark.createDataFrame(summary_p)

In [47]:
summaryT.show()

+------------+-------+-------------------+-------------------+---+------+
|       field|  count|               mean|             stddev|min|   max|
+------------+-------+-------------------+-------------------+---+------+
|        id_1|5749132|  33324.48559643438| 23659.859374487987|  1| 99980|
|        id_2|5749132|  66587.43558331935| 23620.487613269706|  6|100000|
|cmp_fname_c1|5748125| 0.7129024704425707| 0.3887583596162788|0.0|   1.0|
|cmp_fname_c2| 103698|  0.900017671890335| 0.2713176105782331|0.0|   1.0|
|cmp_lname_c1|5749132|0.31562781930763056| 0.3342336339615803|0.0|   1.0|
|cmp_lname_c2|   2464|0.31841283153174366| 0.3685670662006655|0.0|   1.0|
|     cmp_sex|5749132|  0.955001381078048|0.20730111116897443|  0|     1|
|      cmp_bd|5748337|0.22446526708507172|0.41722972238461925|  0|     1|
|      cmp_bm|5748337|0.48885529849763504| 0.4998758236779003|  0|     1|
|      cmp_by|5748337| 0.2227485966810923| 0.4160909629831711|  0|     1|
|     cmp_plz|5736289|0.00552866147434

In [48]:
summaryT.printSchema()

root
 |-- field: string (nullable = true)
 |-- count: string (nullable = true)
 |-- mean: string (nullable = true)
 |-- stddev: string (nullable = true)
 |-- min: string (nullable = true)
 |-- max: string (nullable = true)



In [49]:
from pyspark.sql.types import DoubleType


for column in summaryT.columns:
  if column == "field":
    continue
  summaryT = summaryT.withColumn(column, summaryT[column].cast(DoubleType()))

In [50]:
summaryT.printSchema()

root
 |-- field: string (nullable = true)
 |-- count: double (nullable = true)
 |-- mean: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- min: double (nullable = true)
 |-- max: double (nullable = true)



In [51]:
from pyspark.sql import DataFrame
from pyspark.sql.types import DoubleType


def pivot_summary(desc):
  # Convert to Pandas dataframe
  desc_p = desc.toPandas()

  # Transpose
  desc_p = desc_p.set_index("summary").transpose().reset_index()
  desc_p = desc_p.rename(columns={"index":"field"})
  desc_p = desc_p.rename_axis(None, axis=1)

  # Convert to Spark DataFrame
  descT = spark.createDataFrame(desc_p)

  # Convert metric columns to double from string
  for column in descT.columns:
    if column == "field":
      continue
    else:
      descT = descT.withColumn(column, descT[column].cast(DoubleType()))
    return descT

In [52]:
match_summaryT = pivot_summary(match_summary)

In [57]:
miss_summaryT = pivot_summary(miss_summary)

In [58]:
match_summaryT.createOrReplaceTempView("match_desc")
miss_summaryT.createOrReplaceTempView("miss_desc")

spark.sql("""
  SELECT a.field, a.count + b.count AS total, a.mean - b.mean AS delta
  FROM match_desc AS a
  INNER JOIN miss_desc AS b
  ON a.field = b.field
  WHERE a.field NOT IN ("id_1", "id_2")
  ORDER BY delta DESC, total DESC
""").show()

+------------+---------+--------------------+
|       field|    total|               delta|
+------------+---------+--------------------+
|     cmp_plz|5736289.0|  0.9563812499852176|
|cmp_lname_c2|   2464.0|  0.8064147192926263|
|      cmp_by|5748337.0|  0.7762059675300512|
|      cmp_bd|5748337.0|   0.775442311783404|
|cmp_lname_c1|5749132.0|  0.6838772482598161|
|      cmp_bm|5748337.0|  0.5109496938298685|
|cmp_fname_c1|5748125.0| 0.28545290574713356|
|cmp_fname_c2| 103698.0| 0.09104268062278553|
|     cmp_sex|5749132.0|0.032408185250332844|
+------------+---------+--------------------+



In [59]:
good_features = ["cmp_lname_c1", "cmp_plz", "cmp_by", "cmp_bd", "cmp_bm"]

In [60]:
sum_expression = " + ".join(good_features)

In [61]:
sum_expression

'cmp_lname_c1 + cmp_plz + cmp_by + cmp_bd + cmp_bm'

In [63]:
from pyspark.sql.functions import expr


scored = parsed.fillna(0, subset=good_features).\
                withColumn("score", expr(sum_expression)).\
                select("score", "is_match")

In [64]:
scored.show()

+-----+--------+
|score|is_match|
+-----+--------+
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
+-----+--------+
only showing top 20 rows



In [67]:
def crossTabs(socred: DataFrame, t: DoubleType) -> DataFrame:
  return scored.selectExpr(f"score >= {t} AS above", "is_match").\
         groupBy("above").pivot("is_match", ("true", "false")).\
         count()

In [68]:
crossTabs(scored, 4.0).show()

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20871|    637|
|false|   60|5727564|
+-----+-----+-------+



In [69]:
crossTabs(scored, 2.0).show()

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20931| 596414|
|false| NULL|5131787|
+-----+-----+-------+

